# Introduction
These functions are used to obtain the data from the server using the Obspy package.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from obspy.taup import TauPyModel
from obspy.geodetics import gps2dist_azimuth, kilometer2degrees
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

import pickle
import copy

Let us define a retrieval function to be used for getting the data

In [2]:
def getEvents(clientName, networkName,  stationName, startTime, endTime, minMagnitude, maxRadius):
    '''
    Obtain a catalog of the seismic events around the base station
    '''
    # Find the possible earthquakes and put them in a catalog
    client = Client(clientName)
    inventory = client.get_stations(network=networkName)# , station=stationName)
    station = inventory[0][0]
    cat = client.get_events(starttime=startTime, endtime=endTime, minmagnitude=minMagnitude, latitude=station.latitude, longitude=station.longitude, maxradius=maxRadius)
    
    return cat

def getStreams(clientName, networkName,  stationNames, channelName, cat, timeBefore, timeAfter):
    '''
    Get the streams for the events (the seismic signal)
    '''
    # Get the stream for each event
    model = TauPyModel(model="iasp91")
    client = Client(clientName)

    
    nEvents = len(cat)
    print("Will analyze " + str(nEvents) + " events")
    stList = []
    for i in range(0, nEvents):
        for stationName in stationNames:
            try:
                inventory = client.get_stations(network=networkName, station=stationName)
                station = inventory[0][0]
                event = cat[i]
                origin = event.origins[0]
                distance, _, _ = gps2dist_azimuth(origin.latitude, origin.longitude, station.latitude, station.longitude)
                distance = kilometer2degrees(distance / 1e3)
                arrivals = model.get_travel_times(origin.depth / 1e3, distance)
                traveltime = arrivals[0].time
                arrival_time = origin.time + traveltime
                # Get the earthquake signal and store its needed information
                st = client.get_waveforms(network=networkName, station=stationName, location="00", channel=channelName, starttime=arrival_time-timeBefore, endtime=arrival_time+timeAfter)
                stList.append(st)
                print("Got event " + str(i) + " in station " + stationName)
            except:
                print("Could not get stream " + str(i) + " in station " +  stationName)
    return stList

def getRandomStreams(clientName, networkName,  stationNames, channelName, timeBefore, timeAfter, starttime, endtime, nEvents = 1):
    '''
    Get the streams for the events for random singals (negative labels)
    '''
    client = Client(clientName)
    
    print("Will analyze " + str(nEvents) + " events")
    stList = []
    for i in range(0, nEvents):
        for stationName in stationNames:
            try:
                inventory = client.get_stations(network=networkName, station=stationName)
                station = inventory[0][0]
                arrival_time = startTime + (endTime - startTime) * np.random.random()
                st = client.get_waveforms(network=networkName, station=stationName, location="00", channel=channelNames, starttime=arrival_time-timeBefore, endtime=arrival_time+timeAfter)
                stList.append(st)
                print("Got random event " + str(i) + " in station " + stationName)
            except:
                print("Could not get random stream " + str(i) + " in station " +  stationName)
    return stList

let us make a function that save and load the list of seismogram. It is a list of objects so we have to use the package pickle which is included in python by default.

In [3]:
def saveList(listToSave, fileName = 'SavedList.dat'):
    '''
    Save a list into a file.
    '''
    pickle.dump( listToSave, open(fileName, "wb" ))


def loadList(fileName):
    '''
    Load a list from a file
    '''
    loadedList = pickle.load( open( fileName, "rb" ))
    return loadedList

Finally, we define a function to extract the data to numpy format for further analysis.

In [4]:
def st2numpy(st, traceNumber=0):
    '''
    Retrive information from the seismogram raw object
    '''
    trace = st.pop(traceNumber)
    data = trace.data
    time = trace.times('matplotlib')
    return time, data

def stList2numpy(stList, traceNumber=0, label=1):
    '''
    Retrive information from multiple seismograms and put them in an array
    '''
    nSt = len(stList)
    timeList = []
    dataList = []
    labelList = []
    
    time, data = st2numpy(stList[0], traceNumber)
    timeShape = time.shape
    dataShape = data.shape
    
    timeList.append(time)
    dataList.append(data)
    labelList.append(label)
    
    for i in range(1, nSt):
        time, data = st2numpy(stList[i], traceNumber)
        if (data.shape == dataShape):
            timeList.append(time)
            dataList.append(data)
            labelList.append(label)

    
    timeArray = np.asarray(timeList).T;
    dataArray = np.asarray(dataList).T;
    labelArray = np.asarray(labelList).T;
    labelArray = labelArray[:,np.newaxis]

    return timeArray, dataArray, labelArray
  

# Building the dataset
Now, let us retrieve the dataset and save it. This is the actual script.

In [8]:
networkName      = "BK"                                # Seismic network name
#stationNames      = ["PKD", "BRK", "BDM", "WENL"]      # Station names (first station is the base station)
stationNames      = ["TSCN", "PKD", "THIS", "RAMR", "TCHL"]      # Station names (first station is the base station)

clientName       = "NCEDC"                             # Client name (server)
startTime        = UTCDateTime("2010-06-01")           # Start time to look for data
endTime          = UTCDateTime("2018-05-14")           # End time to look for data 
maxRadius        = .12 # angle                          # Maximum radius (in angles) away from the base station
minMagnitude     = 3                                  # Minimum seismic magnitude for teh event
channelNames      = "BH?"#vertical                    # Channel name (LHZ: 1Hz; BHZ: 40 Hz)
neg2pos = 1;                                          # Approximate negative to positive labels
secWarning = 60                                       # Number of seconds for the warning period (before seismic event)
secPrecurser = 60                                  # Number of seconds for teh seismic precurser signal 

# Obtain the siesmic events 
timeAfter  = -secWarning;     
timeBefore = secWarning+secPrecurser;
cat  = getEvents(clientName, networkName,  stationNames[0], startTime, endTime, minMagnitude, maxRadius)
print(str(len(cat))+ " seismic events found")

3 seismic events found


In [9]:
# get the data from the server
stListPositives = getStreams(clientName, networkName,  stationNames, channelNames, cat, timeBefore, timeAfter)
nNegatives = int(neg2pos * len(stListPositives) / len(stationNames))
stListNegatives = getRandomStreams(clientName, networkName,  stationNames, channelNames, timeBefore, timeAfter, startTime, endTime, nNegatives)

# save it in raw form (object: stream)
attributeString = "_M_" + str(minMagnitude) + "_R_" + str(maxRadius) +  "_S_" + str(len(stationNames)) + "_Sec_" + str(secPrecurser)
saveList(stListPositives, fileName = "PositivesList_Test_" + attributeString +  ".dat")
saveList(stListNegatives, fileName = "NegativeList_Test_" + attributeString +  ".dat")

Will analyze 3 events
Got event 0 in station TSCN
Got event 0 in station PKD
Got event 0 in station THIS
Got event 0 in station RAMR
Got event 0 in station TCHL
Got event 1 in station TSCN
Got event 1 in station PKD
Got event 1 in station THIS
Got event 1 in station RAMR
Could not get stream 1 in station TCHL
Could not get stream 2 in station TSCN
Got event 2 in station PKD
Could not get stream 2 in station THIS
Got event 2 in station RAMR
Could not get stream 2 in station TCHL
Will analyze 2 events
Got random event 0 in station TSCN
Got random event 0 in station PKD
Got random event 0 in station THIS
Could not get random stream 0 in station RAMR
Got random event 0 in station TCHL
Got random event 1 in station TSCN
Got random event 1 in station PKD
Got random event 1 in station THIS
Got random event 1 in station RAMR
Got random event 1 in station TCHL


In [ ]:
stListPositives[0]
timeArrayNeg, dataArrayNeg, labelArrayNeg = stList2numpy(copy.deepcopy(stListNegatives),0, label = 0)
timeArrayPos, dataArrayPos, labelArrayPos = stList2numpy(copy.deepcopy(stListPositives),0, label = 1)

In [18]:
time.shape

(2400, 20)

In [19]:
# Convert the streams into numpy arrays
timeArrayNeg1, dataArrayNeg1, labelArrayNeg1 = stList2numpy(copy.deepcopy(stListNegatives),0, label = 0)
timeArrayPos1, dataArrayPos1, labelArrayPos1 = stList2numpy(copy.deepcopy(stListPositives),0, label = 1)

timeArrayNeg2, dataArrayNeg2, labelArrayNeg2 = stList2numpy(copy.deepcopy(stListNegatives),1, label = 0)
timeArrayPos2, dataArrayPos2, labelArrayPos2 = stList2numpy(copy.deepcopy(stListPositives),1, label = 1)

timeArrayNeg3, dataArrayNeg3, labelArrayNeg3 = stList2numpy(copy.deepcopy(stListNegatives),2, label = 0)
timeArrayPos3, dataArrayPos3, labelArrayPos3 = stList2numpy(copy.deepcopy(stListPositives),2, label = 1)

# Concatinate positive and negative arrays
#timeArrayNeg  = np.concatenate([timeArrayNeg1 , timeArrayNeg2, timeArrayNeg3], axis = 2);
#timeArrayPos  = np.concatenate([timeArrayPos1 , timeArrayPos2, timeArrayPos3], axis = 2);

dataArrayNeg  = np.stack([dataArrayNeg1 , dataArrayNeg2, dataArrayNeg3], axis = 2);
dataArrayPos  = np.stack([dataArrayPos1 , dataArrayPos2, dataArrayPos3], axis = 2);

#labelArrayNeg  = np.concatenate([labelArrayNeg1 , labelArrayNeg2, labelArrayNeg3], axis = 2);
#labelArrayPos  = np.concatenate([labelArrayPos1 , labelArrayPos2, labelArrayPos3], axis = 2);


# Concatinate positive and negative arrays
time  = np.concatenate([timeArrayPos1 , timeArrayNeg1], axis = 1);
data  = np.concatenate([dataArrayPos , dataArrayNeg], axis = 1);
label = np.concatenate([labelArrayPos1 , labelArrayNeg1]);

# Save numpy arrays for usage in the neural network
np.save("TimeTest" + attributeString + ".npy", time)
np.save("DataTest" + attributeString + ".npy", data)
np.save("LabelTest" + attributeString + ".npy", label)

## Just to make sure, Let's plot some data

In [ ]:
i = 3
fig = plt.figure()
plt.plot(timeArrayNeg[:,i], dataArrayNeg[:,i])
fig = plt.figure()
plt.plot(timeArrayPos[:,i], dataArrayPos[:,i])

## Calculate the spectogram

In [ ]:
# These functions are based on Obspy internal functions

import math as M
from matplotlib import mlab
from matplotlib.colors import Normalize
from obspy.imaging.cm import obspy_sequential
import matplotlib.pyplot as plt
from skimage.transform import resize

def getSpectogram(data, samp_rate, per_lap=0.9, wlen=None, log=False,
                outfile=None, fmt=None, axes=None, dbscale=False,
                mult=8.0, zorder=None, title=None,
                show=True, sphinx=False, clip=[0.0, 1.0]):
    
    # enforce float for samp_rate
    samp_rate = float(samp_rate)

    # set wlen from samp_rate if not specified otherwise
    if not wlen:
        wlen = samp_rate / 100.

    npts = len(data)
    # nfft needs to be an integer, otherwise a deprecation will be raised
    # XXX add condition for too many windows => calculation takes for ever
    nfft = int(_nearest_pow_2(wlen * samp_rate))
    if nfft > npts:
        nfft = int(_nearest_pow_2(npts / 8.0))

    if mult is not None:
        mult = int(_nearest_pow_2(mult))
        mult = mult * nfft
    nlap = int(nfft * float(per_lap))

    data = data - data.mean()
    end = npts / samp_rate

    
    specgram, freq, time = mlab.specgram(data, Fs=samp_rate, NFFT=nfft,
                                         pad_to=mult, noverlap=nlap)
    # db scale and remove zero/offset for amplitude
    if dbscale:
        specgram = 10 * np.log10(specgram[1:, :])
    else:
        specgram = np.sqrt(specgram[1:, :])
    freq = freq[1:]

    vmin, vmax = clip
    if vmin < 0 or vmax > 1 or vmin >= vmax:
        msg = "Invalid parameters for clip option."
        raise ValueError(msg)
    _range = float(specgram.max() - specgram.min())
    vmin = specgram.min() + vmin * _range
    vmax = specgram.min() + vmax * _range
    norm = Normalize(vmin, vmax, clip=True)
    
    return specgram, freq, time


def _nearest_pow_2(x):
    """
    Find power of two nearest to x

    >>> _nearest_pow_2(3)
    2.0
    >>> _nearest_pow_2(15)
    16.0

    :type x: float
    :param x: Number
    :rtype: Int
    :return: Nearest power of 2 to x
    """
    a = M.pow(2, M.ceil(np.log2(x)))
    b = M.pow(2, M.floor(np.log2(x)))
    if abs(a - x) < abs(b - x):
        return a
    else:
        return b

In [ ]:
# Get some data and plot the spectogram
data  = np.load("Datasets\Data_M_2.8_R_0.5_S_4_Sec_256.npy")
label = np.load("Datasets\Label_M_2.8_R_0.5_S_4_Sec_256.npy")
time  = np.load("Datasets\Time_M_2.8_R_0.5_S_4_Sec_256.npy")

data  = np.load("Datasets\DataExamples.npy")
label = np.load("Datasets\LabelExamples.npy")
time  = np.load("Datasets\TimeExamples.npy")

i = 3

specgram, freq, time = getSpectogram(data[:,i], 40)
specgram = np.log10(specgram)
newSize = np.round(np.array(specgram.shape) *.1)
#specgram = resize(specgram,[64,64])

fig = plt.figure(num=None, figsize=(5, 4), dpi=100)
plt.imshow(specgram, extent=[np.min(time),np.max(time)    ,np.max(freq),np.min(freq)])
plt.gca().set_aspect(10)
plt.gca().invert_yaxis()



In [ ]:
# Another way of plotting the spectogram (using the built in function of Obspy)
stListPositives  = np.load("Raw Data\PositivesList__M_2.8_R_0.5_S_4_Sec_256.dat")
stListPositives[1].spectrogram()

